In [11]:
import numpy as np
import os
from os import listdir
from os.path import isfile, join
import pickle
from tqdm import tqdm

path = 'D:/Datasets/Motion Privacy/NTU RGB+D 120/Skeleton Data'
frame_time = 0.03333333 # 30 fps

In [3]:
with open('template.bvh', 'rb') as f:
    template = f.read()

In [14]:
# Attempt to load X and Y from pickle before generating them
def load_files():
    # Read the files
    files = [f for f in listdir(path) if isfile(join(path, f))]

    # Get stats for each file based on name
    files_ = []
    for file in files:
        data = {'file': file,
                's': file[0:4],
                'c': file[4:8],
                'p': file[8:12],
                'r': file[12:16],
                'a': file[16:20]
                }
        files_.append(data)

    # Generate X and Y
    for file_ in tqdm(files_, desc='Files Parsed', position=0):
        try:
            file = join(path, file_['file'])
            data = open(file, 'r')
            lines = data.readlines()
            frames_count = int(lines.pop(0).replace('\n', ''))
            file_['frames'] = frames_count
        except UnicodeDecodeError: # .DS_Store file
            print('UnicodeDecodeError: ', file)
            continue

        # Get P and File
        p = file_['p']
        file = file_['file']
        
        # Skip file if 2 actors
        if lines[0].replace('\n', '') != '1': continue
        multi_actor = False

        # Get frame data
        frame_values = []
        for f in tqdm(range(frames_count), desc='Frames Parsed', position=1, leave=False):
            try:
                # Get actor count
                actors = int(lines.pop(0).replace('\n', ''))
                if actors > 1: 
                    multi_actor = True
                    break
                # Get actor info
                t = lines.pop(0)

                # Get joint count
                joint_count = int(lines.pop(0).replace('\n', ''))

                # Get joint info
                frame_value = []
                for j in range(joint_count):
                    joint = lines.pop(0).replace('\n', '').split(' ')
                    frame_value.extend(joint[0:3])
                    frame_value.extend(joint[8:11])

                frame_values.append(frame_value)
            except:
                break
        
        if multi_actor: continue

        # Format file
        frames = len(frame_values)
        bvh = template.decode('utf-8')
        bvh = bvh + 'Frames: ' + str(frames)
        bvh = bvh + '\nFrame Time: ' + str(frame_time)
        for frame_value in frame_values:
            bvh = bvh + '\n' + ' '.join(frame_value)
        
        # Save file
        if not os.path.exists(f'./data/Skeleton Data BVH/{p}'):
            os.makedirs(f'./data/Skeleton Data BVH/{p}')
            
        with open(f'./data/Skeleton Data BVH/{p}/{file}.bvh', 'wb') as f:
            f.write(bvh.encode('utf-8'))

        break # Only do one file for now
load_files()

Files Parsed:   0%|          | 0/114480 [00:00<?, ?it/s]
